# Launch Docker Instance

```
docker run --name typedb -d -v typedb-data:/opt/ -p 1729:1729 --platform linux/amd64 vaticle/typedb:latest
```

# Check Connectivity

In [ ]:
from typedb.driver import *
client = TypeDB.core_driver("192.168.1.170:1729")
client.close()

# Quickstart
(from https://typedb.com/docs/drivers/python/overview)

In [ ]:
DB_NAME = "typedb-tutorial-db"
SERVER_ADDR = "192.168.1.170:1729"

with TypeDB.core_driver(SERVER_ADDR) as driver:
    if driver.databases.contains(DB_NAME):
        driver.databases.get(DB_NAME).delete()
    driver.databases.create(DB_NAME)

    with driver.session(DB_NAME, SessionType.SCHEMA) as session:
        with session.transaction(TransactionType.WRITE) as tx:
            tx.query.define("define person sub entity;")
            tx.query.define("define name sub attribute, value string; person owns name;")
            tx.commit()

    with driver.session(DB_NAME, SessionType.DATA) as session:
        with session.transaction(TransactionType.WRITE) as tx:
            tx.query.insert("insert $p isa person, has name 'Alice';")
            tx.query.insert("insert $p isa person, has name 'Bob';")
            tx.commit()
        with session.transaction(TransactionType.READ) as tx:
            results = tx.query.fetch("match $p isa person; fetch $p: name;")
            for json in results:
                print(json)

# Test LEDHNTR Client

In [ ]:
from pprint import pprint, pformat
from ledhntr import LEDHNTR
from ledhntr.data_classes import(
    Attribute, Entity, Relation, Thing
)

from pkg_resources import resource_stream

led = LEDHNTR()
tdb = led.plugins['typedb_client']
DB_NAME = "typedb-tutorial-db"
SCHEMA = resource_stream('ledhntr', 'schemas/schema.tql').name
ROAD_SCHEMA = resource_stream('ledhntr', 'schemas/road.tql').name

tdb.check_db(DB_NAME)

## Get all DBs

In [ ]:
dbs = tdb.get_all_dbs()
all_dbs = []
for dbo in dbs:
    db = str(dbo)
    if db not in all_dbs:
        all_dbs.append(db)
        print(db)

## Search DB for results

In [ ]:
from pprint import pprint, pformat
tdb.db_name = DB_NAME
alice = Entity(label='person', has=[Attribute(label='name', value='Alice')])
res = tdb.find_things(alice)
pprint(res)
pprint(res[0].to_dict())

## Bootstrap ROAD (Root of all data)

In [ ]:
def _bootstrap_road(dbc):

    # Start with a fresh RoAD DB
    dbc.create_db()
    dbc.write_tql_file(
        file = ROAD_SCHEMA
    )

    # Create base structure
    things = {'attributes':[], 'entities':[], 'relations':[]}
    base_ents = ['actors', 'archives', 'index', 'scratchpad', 'news', 'active-hunts']
    for be in base_ents:
        things['entities'].append(Entity(label=be))
        road = Relation(label='road', players={'related': things['entities']})
    things['relations'].append(road)
    dbc.bulk_add(things, force=True)

    return True

tdb.db_name = 'road'
_bootstrap_road(tdb)

# Generate Dummy Data

In [ ]:
import random
from datetime import datetime, timedelta
from pprint import pprint, pformat
from time import time

def gen_ips(num, existing_ips=[]):
    for i in range(num):
        ip = ".".join(str(random.randint(0,255)) for _ in range(4))
        if ip not in existing_ips:
            existing_ips.append(ip)
    if len(existing_ips) >= num:
        return existing_ips
    missing = num-len(existing_ips)
    if missing:
        print(f"Missing {missing} IPs - generating more...")
    existing_ips = gen_ips(missing, existing_ips)
    return existing_ips

def gen_lorem(words, num_sentences=1):
    sentences = []
    for s in range(num_sentences):
        lorem_ipsum_text = (
            "Lorem ipsum dolor sit amet consectetur adipiscing elit sed do eiusmod tempor incididunt ut labore et dolore "
            "magna aliqua Ut enim ad minim veniam quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo "
            "consequat Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur "
            "Excepteur sint occaecat cupidatat non proident sunt in culpa qui officia deserunt mollit anim id est laborum"
        )
        words_list = lorem_ipsum_text.split()
        output_text = ' '.join(random.choice(words_list) for _ in range(words))
        output_text += "."
        sentences.append(f"{output_text}")
    return ' '.join(sentences)

def roll_die(sides=6):
    return random.randint(1,sides)

def gen_hunts(num_hunts=1, num_ips=50):
    all_hunts = []
    for nh in range(num_hunts):
        ips = gen_ips(num_ips)
        now = datetime.now()
        yesterday = now - timedelta(days=1)
        tomorrow = now + timedelta(days=1)

        date_discovered = Attribute(label='date-discovered', value=yesterday)
        date_seen_yest = Attribute(label='date-seen', value=yesterday)
        date_seen_today = Attribute(label='date-seen', value=now)
        date_seen_tom = Attribute(label='date-seen', value=tomorrow)
        asn = Attribute(label='note', value='TRASH-PANDA-SERVERS')

        ipents = []

        for ip in ips:
            ipattr = Attribute(label='ip-address', value=ip)
            ipent = Entity(label='ip', has=[ipattr, date_discovered, date_seen_yest, date_seen_today, date_seen_tom])
            roll = roll_die(3)
            if roll==3:
                ipent.has.append(asn)
            else:
                randasn = gen_lorem(3).replace(' ', '-').upper()
                randasnattr = Attribute(label='note', value=randasn)
                ipent.has.append(randasnattr)
            ipents.append(ipent)

        nowint = int(time())
        hunt_name = Attribute(label='hunt-name', value=f'HUNT-{nh}-{nowint}')
        hunt_active = Attribute(label='hunt-active', value=True)
        hunt = Relation(label='hunt', has=[hunt_name, hunt_active], players={'related': ipents})
        # pprint(hunt)
        # pprint(hunt.players['related'][0].to_dict())
        all_hunts.append(hunt)
    return all_hunts


## Add Dummy Data To Database

In [ ]:
from ledhntr import LEDHNTR
from time import time
from ledhntr.data_classes import(
    Attribute, Entity, Relation, Thing
)

from importlib.resources import files

led = LEDHNTR()
tdb = led.plugins['typedb_client']
DB_NAME = "DUMMY-DATA-DB"
tdb.db_name = DB_NAME
SCHEMA = str(files('ledhntr').joinpath('schema/schema.tql'))
nukeit=False

if nukeit:
    tdb.delete_db(DB_NAME)

if not tdb.check_db(DB_NAME):
    tdb.db_name = DB_NAME
    tdb.create_db()
    tdb.write_tql_file(file = SCHEMA)

# tdb.add_relation(hunt) # @ This took 5 seconds to write everything

# @ This was way faster, but I don't think it did it right.
# things = {'attributes': [], 'entities': [], 'relations': [hunt]}
# tdb.bulk_add(things)
# led.logger.setLevel('DEBUG')
all_hunts = gen_hunts(num_hunts=2)
stime = time()
led.logger.info(f"START ADD RELATION {stime}")
tdb.add_relation(all_hunts[0])
etime = time()
led.logger.info(f"END ADD RELATION {etime} - Length: {etime-stime}")
stime = time()
led.logger.info(f"START BULK_ADD {stime}")
things = {'attributes': [], 'entities': [], 'relations': [all_hunts[1]]}
tdb.bulk_add(things)
etime = time()
led.logger.info(f"END BULK_ADD {etime} - Length: {etime-stime}")
# led.logger.setLevel('INFO')

In [ ]:
tdb.db_name = "DUMMY-DATA-DB"
all_hunts = gen_hunts(num_hunts=2)
for ah in all_hunts:
    print(ah)



In [ ]:
so = Relation(label='hunt', has=[Attribute(label='hunt-name', value='HUNT-0-1715022040')])
res = tdb.find_things(so)
print(res)

In [ ]:
pprint(res[0].players['related'][0].to_dict())

In [ ]:
so = Relation(label='hunt')
tdb.find_things(so)

In [ ]:
# led.logger.setLevel('DEBUG')
ahents = all_hunts[0].players['related']
things = {'attributes': [], 'entities': ahents, 'relations': [all_hunts[0]]}
tdb.bulk_add(things)
# led.logger.setLevel('INFO')

In [ ]:
from ledhntr.data_classes import Query
q = '''$ip_0012c iid 0x826e8002800000000000012c; get $ip_0012c;'''
q = '''$ip_0 isa ip, has ip-address $ip-address_0; $ip-address_0 = "254.98.204.230"; fetch $ip_0 as IP: ip-address;'''
q = '''$ip isa ip, has ip-address $ip1; $ip1 = '254.98.204.230'; get $ip;'''

MyQuery = Query(qtype='match', string=q)
# led.logger.setLevel('DEBUG')
final_answers = tdb.raw_query(MyQuery)
pprint(final_answers)
# led.logger.setLevel('INFO')

In [ ]:
tdb.db_name = "DUMMY-DATA-DB"
ip_attr = Attribute(label='ip-address', value='254.98.204.230')
ipent = Entity(label='ip', has=[ip_attr])
pprint(tdb.get_query_from_thing(ipent))

# tdb.find_things(ipent)